In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from functions.plot_data import plot_data
from functions.visib import visib
from functions.prob_city import prob_city
from functions.tau_branch import tau_branch
from functions.tau_prune import tau_prune
from Ant import Ant
from Colony import Colony
from functions.tsp import tsp
from functions.colSelection import colSelection
from functions.colCreate import colCreate
from functions.fitness import fitness
from functions.colMutate import colMutate
from functions.colEvo import colEvo

In [3]:
# Get visibility matrix
n_mat = visib()

# Get the number of cities
N = len(n_mat[0,:])

''' Ants '''
# Define number of ants and list object that will hold them
number_of_ants = N


# Define global ants characteristics
#alpha = 1 # Pheromone effect
beta = 5 # Greed
Q = 100
init_city = 0

''' Stochastic Options'''
# Define random characteristics
random.seed(1)

''' Initialization '''
iters = 50 # Number of ant cycles
nColonies = 2
alpha_min = 0.99
alpha_max = 1.01
d_alpha_max = 0.01 # Max evo shift in alpha
# Fitness function parameters
c0 = 0
c1 = 1000

In [4]:
# Initialize a list of ant colonies
colony_list = colCreate(nColonies, number_of_ants, N, alpha_min,
                        alpha_max, beta, Q)

In [5]:
# Cycle each colony through TSP 'iters' times
for colony in colony_list:
    colony = tsp(colony, iters, n_mat)

    # Update minimum and total paths for the colony
    colony.getMinL()
    colony.getTotL()

In [6]:
for colony in colony_list:
    print(colony.minL)
    print(colony.L)

0
796979.381118
0
779764.62061
0
781106.002136
0
781972.527614
0
765573.826104


In [5]:
# Colony selection initialization
col_list = colony_list
nCols = len(col_list)
fitness_list = []

In [14]:
# Play with built in colony evolution method
random.seed(1)
col = col_list[1]
print(col.getAlphaDist())
#plt.hist(col.getAlphaDist())
#plt.show()
col.mutate(d_alpha_max)
print(col.getAlphaDist())
#plt.hist(col.getAlphaDist())
#plt.show()
print(col.minL)

[ 1.00771725  1.17030557  0.78415937  0.99936206  0.89335362  0.53452583
  1.24373066  0.914314    0.86712383  1.20304076  1.23821546  0.93896163
  0.60793126  1.02093842  0.87052267  0.52957496  0.80852943  1.48318772
  1.07817591  0.6703492   0.72693735  1.27052314  1.42008643  0.73217613
  0.84382589  1.07779481  1.40975502  1.04799631  0.50570913  1.35694913]
[ 0.97115367  1.20504894  0.81053683  0.97486896  0.89289713  0.52947494
  1.25888995  0.94318633  0.82650979  1.15587551  1.27179197  0.93223834
  0.63415927  0.97114902  0.86506139  0.55172897  0.78140565  1.52771479
  1.11831865  0.6234082   0.67948193  1.27466439  1.46400135  0.72029655
  0.81548583  1.07000647  1.36265909  1.02016548  0.49949789  1.35653035]
442.271142194


In [12]:
# Colony selection - Get fitness list and total fitness
for colony in col_list:
    fitness_list.append(fitness(colony, c0, c1))

    totFit = sum(fitness_list)
    # This array holds votes for each colony in col_list
    colonyVotes = np.zeros(nCols)
print(fitness_list)
print(totFit)

[2.2302271061221748, 2.2610564077013162, 2.2282706461015382, 2.2286031464661713, 2.202563260664899]
11.1507205671


In [13]:
# Colony selection - calculate votes
colonyVotes = np.zeros(nCols)
# Iterate over the total number of new colonies to be created
for j in range(nCols):
    # Assign random number over size of total fitness
    temp1 = random.uniform(0, totFit)
    #print("Temp1 = " + str(temp1))
    # Initialize an accumulator
    tot = 0
    # Get cumulative sum of fitnesses to decide on colony
    for i in range(nCols):
        # Update cumulative sum
        tot = tot + fitness_list[i]
        #print("Tot = " + str(tot))
        # If less than the random number, increase vote
        if temp1<tot:
            colonyVotes[i] = colonyVotes[i] + 1
            break
print(colonyVotes)

[ 0.  4.  1.  0.  0.]


In [11]:
# General program parm initialization
''' Initialization '''
iters = 10 # Number of ant cycles
nColonies = 5
alpha_min = 0.5
alpha_max = 1.5
d_alpha_max = 0.1 # Max evo shift in alpha
evo_cycles = 5
fitness_mat = np.zeros((evo_cycles, nColonies))
# Fitness function parameters
c0 = 0
c1 = 1000

# Initialize a list of ant colonies
colony_list = colCreate(nColonies, number_of_ants, N, alpha_min,
                        alpha_max, beta, Q)

In [12]:
# Run tsp and evolution
for i in range(evo_cycles):
    j = 0
    # Cycle each colony through TSP 'iters' times
    for colony in colony_list:
        colony = tsp(colony, iters, n_mat)

        # Update minimum and total paths for the colony
        colony.getMinL()
        colony.getTotL()
        fitness_mat[i,j]=fitness(colony, c0, c1)
        j = j+1

    #colony_list = colEvo(colony_list, c0, c1, d_alpha_max)

In [39]:
# Get example colony
colony = colony_list[1]
colony.minL

459.78607518776028

In [15]:
offspring_list = colonyVotes
print(offspring_list)

[ 0.  4.  1.  0.  0.]


In [16]:
new_col_list = colEvo(col_list, c0, c1, d_alpha_max)

In [19]:
# Compare new and old colony lists
for colony in col_list:
    print(colony.getAlphaDist())

[ 1.13845005  1.55636417  0.24585982  0.9708198   0.78592815  0.09982696
  1.75183167  0.81444031  1.55966642  0.85732533  1.65676697  1.91171393
  0.67673708  0.06082742  0.04742728  0.79700946  1.85029425  0.11195539
  1.52498943  1.15940494  0.43259857  1.47907436  1.78184218  1.71567958
  1.75122768  0.37923134  1.71817583  1.42108829  1.32393479  0.96984511]
[ 1.0154345   1.34061113  0.56831873  0.99872412  0.78670725  0.06905166
  1.48746131  0.828628    0.73424766  1.40608152  1.47643093  0.87792326
  0.21586252  1.04187684  0.74104533  0.05914993  0.61705885  1.96637543
  1.15635181  0.34069839  0.45387469  1.54104628  1.84017287  0.46435226
  0.68765178  1.15558962  1.81951003  1.09599262  0.01141826  1.71389825]
[ 1.8769184   1.6182798   1.55479436  0.85218136  0.96220363  1.13999867
  1.88202702  0.96985022  0.82884904  1.07695759  1.57288877  0.9162936
  1.60913897  0.35442252  0.36156799  1.59687788  1.70254802  0.51058808
  1.88714051  0.16646828  0.90599757  1.51117355  

In [20]:
for colony in new_col_list:
    print(colony.getAlphaDist())

[ 1.13845005  1.55636417  0.24585982  0.9708198   0.78592815  0.09982696
  1.75183167  0.81444031  1.55966642  0.85732533  1.65676697  1.91171393
  0.67673708  0.06082742  0.04742728  0.79700946  1.85029425  0.11195539
  1.52498943  1.15940494  0.43259857  1.47907436  1.78184218  1.71567958
  1.75122768  0.37923134  1.71817583  1.42108829  1.32393479  0.96984511]
[ 1.13845005  1.55636417  0.24585982  0.9708198   0.78592815  0.09982696
  1.75183167  0.81444031  1.55966642  0.85732533  1.65676697  1.91171393
  0.67673708  0.06082742  0.04742728  0.79700946  1.85029425  0.11195539
  1.52498943  1.15940494  0.43259857  1.47907436  1.78184218  1.71567958
  1.75122768  0.37923134  1.71817583  1.42108829  1.32393479  0.96984511]
[ 1.13845005  1.55636417  0.24585982  0.9708198   0.78592815  0.09982696
  1.75183167  0.81444031  1.55966642  0.85732533  1.65676697  1.91171393
  0.67673708  0.06082742  0.04742728  0.79700946  1.85029425  0.11195539
  1.52498943  1.15940494  0.43259857  1.47907436 

In [47]:
# Not the same!  colEvo isn't doing what I expect.  It's just copying colonies, not mutating
# colEvo
nColonies = len(col_list)
# Get number of offspring for each colony
offspring_list = colSelection(col_list, c0, c1)
print(offspring_list)
new_col_list = []
# Iterate over list of offspring
for i in range(nColonies):
    # Get number of children for colony i
    children = offspring_list[i]
    if int(children)==0:
        continue
    for j in range(int(children)):
        # Append mutated colony i for each number of children
        print("Copying colony " + str(i))
        print(col_list[i].getAlphaDist())
        new_col_list.append(colMutate(col_list[i], d_alpha_max))
        print(new_col_list[-1].getAlphaDist())

[ 2.  2.  0.  1.  0.]
Copying colony 0
[ 1.15407463  1.50874225  0.23649204  0.93487047  0.78136356  0.22767485
  1.89203476  0.90250848  1.46110788  0.8559354   1.53483609  1.77264154
  0.69662373  0.06493303  0.01790401  0.75475206  1.96624065  0.02888951
  1.46455937  1.17121576  0.47426419  1.55543147  1.83773036  1.72532845
  1.76508957  0.3549326   1.78430126  1.51652447  1.45390792  0.98271216]
[ 1.18824568  1.54992539  0.24786043  0.97782752  0.81382873  0.22659549
  1.91834939  0.91420488  1.43146163  0.88752791  1.49269095  1.75704916
  0.73381613  0.04235515  0.02490732  0.71897764  1.95403699  0.05880438
  1.47660626  1.16024652  0.51182543  1.58153057  1.81849754  1.73774433
  1.7249673   0.38794364  1.74911435  1.4691348   1.43722853  1.02870235]
Copying colony 0
[ 1.18824568  1.54992539  0.24786043  0.97782752  0.81382873  0.22659549
  1.91834939  0.91420488  1.43146163  0.88752791  1.49269095  1.75704916
  0.73381613  0.04235515  0.02490732  0.71897764  1.95403699  0.05

In [1]:
import numpy as np
from time import time
alphaTest = np.loadtxt('alphaBestMin2.txt', delimiter=',', skiprows=1)
fitnessTest = np.loadtxt('fitness_mat2.txt', delimiter=',', skiprows=1)
lMinTest = np.zeros((10,10)) # Nonsense to test

In [3]:
from functions.saveFiles import saveFiles
saveFiles(fitnessTest, alphaTest, lMinTest, 100, 5,
              1, 1, 1, 1, int(time()))

In [2]:
from functions.antplot import plotAlphaDist

In [3]:
plotAlphaDist(alphaTest)

In [2]:
from time import time
int(time())

1438924858